In [3]:
! mkdir -p ../data/laplacian/

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as ss


def sparse_matrix_from_edgelist(edges):
    n = edges["dest"].max()
    ones = np.ones(len(edges["from"]))
    return ss.coo_matrix((ones, (edges["from"], edges["dest"])), shape=(n+1, n+1))
    

def laplacian_embedding(g, dim):
    L = ss.csgraph.laplacian(g)
    w, v = ss.linalg.eigsh(L, k=dim+1)
    return np.divide(v, np.sqrt(w))[::-1][:,1:]

In [2]:
edges = pd.read_parquet('../data/enwiki/pagelinks')
g = sparse_matrix_from_edgelist(edges)

In [4]:
%time ss.save_npz("../data/laplacian/pagelinks.npz", g)

CPU times: user 1min 50s, sys: 1.32 s, total: 1min 51s
Wall time: 1min 52s


In [14]:
%time emb = laplacian_embedding(g, 1)

CPU times: user 10min 55s, sys: 57.4 s, total: 11min 52s
Wall time: 7min 41s


In [20]:
%time np.savetxt(f"../data/laplacian/emb_1.gz", emb)

CPU times: user 6min 31s, sys: 974 ms, total: 6min 32s
Wall time: 6min 33s


In [22]:
! tree -h ../data/laplacian

../data/laplacian
├── [638M]  emb_1.gz
└── [1.5G]  pagelinks.npz

0 directories, 2 files


In [23]:
def run_test(g, k):
    emb = laplacian_embedding(g, k)
    np.savetxt(f"../data/laplacian/emb_{k}.gz", emb)
    return emb

In [24]:
%time run_test(g, 2)

CPU times: user 31min 6s, sys: 1min 12s, total: 32min 18s
Wall time: 24min 2s


array([[ 1.12383496e-14, -5.66814596e-15],
       [ 3.80594743e-15, -2.34071284e-15],
       [ 4.49678668e-15, -2.76558899e-15],
       ...,
       [ 7.14914958e-15, -4.39683062e-15],
       [-1.74306009e-15,  1.07200722e-15],
       [ 9.71373631e-15, -5.97408864e-15]])

In [25]:
%time run_test(g, 4)

CPU times: user 44min 54s, sys: 1min 23s, total: 46min 18s
Wall time: 35min 58s


array([[ 8.09708331e-15,  2.34618574e-14,  4.67226450e-15,
        -4.19850718e-15],
       [ 1.70524025e-15, -7.04938928e-15, -2.39184747e-15,
         2.08077054e-15],
       [-9.88550327e-15,  4.08662421e-14,  1.38658561e-14,
        -1.20625020e-14],
       ...,
       [-1.18396354e-14,  4.89445396e-14,  1.66068107e-14,
        -1.44469757e-14],
       [ 9.95332570e-15, -4.11466171e-14, -1.39609870e-14,
         1.21452604e-14],
       [ 2.98887147e-16, -1.23558651e-15, -4.19232695e-16,
         3.64708474e-16]])

In [26]:
%time run_test(g, 8)

CPU times: user 1h 14min 2s, sys: 1min 38s, total: 1h 15min 41s
Wall time: 1h 19s


array([[ 2.58733259e-14,  1.34176796e-14, -6.54439534e-14, ...,
        -5.10325611e-16, -7.13704947e-16,  7.51289164e-15],
       [-2.73411125e-15,  3.09393578e-15,  1.10286341e-15, ...,
         2.18763309e-15, -5.68043398e-17,  1.45293272e-15],
       [-5.21313345e-16,  5.89921133e-16,  2.10283108e-16, ...,
         4.17116282e-16, -1.08308908e-17,  2.77030869e-16],
       ...,
       [ 1.34614694e-15, -1.52330749e-15, -5.42997727e-16, ...,
        -1.07708696e-15,  2.79677676e-17, -7.15355282e-16],
       [-2.71071381e-14,  3.06745910e-14,  1.09342554e-14, ...,
         2.16891219e-14, -5.63182308e-16,  1.44049910e-14],
       [ 1.50696998e-14, -1.70529576e-14, -6.07869210e-15, ...,
        -1.20576563e-14,  3.13090533e-16, -8.00818178e-15]])

In [27]:
%time run_test(g, 16)

CPU times: user 2h 21min 59s, sys: 32min 48s, total: 2h 54min 48s
Wall time: 2h 14min 59s


array([[-4.03404826e-14,  4.77209790e-14,  9.13477588e-14, ...,
        -1.20514478e-15, -3.01123577e-14,  1.77961209e-14],
       [ 6.56808935e-14, -2.96418294e-14, -1.88610746e-14, ...,
         2.17239843e-14, -2.11093229e-14,  8.74395103e-15],
       [-3.20441325e-15,  1.44615376e-15,  9.20186590e-16, ...,
        -1.05986109e-15,  1.02987323e-15, -4.26596398e-16],
       ...,
       [-6.41376782e-14,  2.89453754e-14,  1.84179214e-14, ...,
        -2.12135652e-14,  2.06133457e-14, -8.53850622e-15],
       [-6.47735708e-14,  2.92323541e-14,  1.86005258e-14, ...,
        -2.14238870e-14,  2.08177166e-14, -8.62316118e-15],
       [-2.58278667e-14,  1.16561328e-14,  7.41678888e-15, ...,
        -8.54257824e-15,  8.30087335e-15, -3.43840635e-15]])

In [28]:
! tree -h ../data/laplacian

../data/laplacian
├── [638M]  emb_1.gz
├── [ 10G]  emb_16.gz
├── [1.3G]  emb_2.gz
├── [2.5G]  emb_4.gz
├── [5.1G]  emb_8.gz
└── [1.5G]  pagelinks.npz

0 directories, 6 files
